In [1]:
import socket
import pickle
import sklearn
import time
from ctypes import create_string_buffer, addressof  
import numpy as np
from struct import pack, unpack

In [2]:
dtree = '/home/admin2/ML_NIC/dtree/cicids/model.pkl'

In [3]:
# Load up model
with open(dtree, 'rb') as f:
    clf = pickle.load(f)

In [4]:
SO_ATTACH_FILTER = 26    
ETH_P_ALL = 3

lines = [
    ( 0x20, 0, 0, 0x00000002 ),
    ( 0x15, 0, 22, 0x4d1379ac ),
    ( 0x28, 0, 0, 0x00000000 ),
    ( 0x15, 0, 20, 0x00000015 ),
    ( 0x20, 0, 0, 0x00000008 ),
    ( 0x15, 0, 18, 0xeba6de3d ),
    ( 0x28, 0, 0, 0x00000006 ),
    ( 0x15, 0, 16, 0x000008c0 ),
    ( 0x28, 0, 0, 0x0000000c ),
    ( 0x15, 13, 0, 0x00004d49 ),
    ( 0x15, 0, 4, 0x000086dd ),
    ( 0x30, 0, 0, 0x00000014 ),
    ( 0x15, 0, 11, 0x00000011 ),
    ( 0x28, 0, 0, 0x00000036 ),
    ( 0x15, 8, 9, 0x0000138d ),
    ( 0x15, 0, 8, 0x00000800 ),
    ( 0x30, 0, 0, 0x00000017 ),
    ( 0x15, 0, 6, 0x00000011 ),
    ( 0x28, 0, 0, 0x00000014 ),
    ( 0x45, 4, 0, 0x00001fff ),
    ( 0xb1, 0, 0, 0x0000000e ),
    ( 0x48, 0, 0, 0x0000000e ),
    ( 0x15, 0, 1, 0x0000138d ),
    ( 0x6, 0, 0, 0x00040000 ),
    ( 0x6, 0, 0, 0x00000000 ),
]

In [5]:
filters_list = []
for l in lines:
    filters_list.append(pack("HBBI", l[0], l[1], l[2], l[3]))

filters = b''.join(filters_list)  
b = create_string_buffer(filters)  
mem_addr_of_filters = addressof(b)  
fprog = pack('HL', len(filters_list), mem_addr_of_filters)  

In [6]:
sock = socket.socket(socket.AF_PACKET, socket.SOCK_RAW, socket.htons(ETH_P_ALL))
sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEPORT, 1)
sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
sock.setsockopt(socket.SOL_SOCKET, SO_ATTACH_FILTER, fprog)
sock.bind(('enp2s0np1', 0))

In [7]:
times_1 = []
times_2 = []

In [ ]:

# Run a dummy to take out the total load and dump time
while (True):
    data, addr = sock.recvfrom(2048)
    
#     start_1 = time.time_ns()
    ethHdr = data[0:14]
    ethH = unpack('!6s6s2s', ethHdr)
    
    ipHdr = data[14:34]
    ipH = unpack('!12s4s4s', ipHdr)
    
#     etherType = int.from_bytes(ethH[2])
    ipProto = ipHdr[9]
    ipFlags = ipHdr[6] >> 5
    tcpDst = 0
    udpDst = 0
    pktSize = int.from_bytes(ipHdr[2:4])
    tcpFlags = 0
    
    l4Hdr = None
    if (ipProto == 6):
        l4Hdr = data[34:54]
        tcpDst = int.from_bytes(l4Hdr[2:4])
        tcpFlags = l4Hdr[13] & 0x3f
        
    elif (ipProto == 17):
        l4Hdr = data[34:42]
        udpDst = int.from_bytes(l4Hdr[2:4])
        
    elif (ipProto == 1 or ipProto == 2):
        l4Hdr = data[34:38]
        
    
    # feature_names = ['etherType', 'ipProto', 'ipFlags', 'tcpDst', 'udpDst', 'pktSize', 'tcpFlags']
    dataIn = np.array([[2048, ipProto, ipFlags, tcpDst, udpDst, pktSize, tcpFlags]])
#     end_1 = time.time_ns()
    
    pred = clf.predict(dataIn)
        
#     start_2 = time.time_ns()
    pred = int(pred[0]).to_bytes(1)
    out_pkt = ethH[1] + ethH[0] + ethH[2] + ipH[0] + ipH[2] + ipH[1] + l4Hdr + pred
#     end_2 = time.time_ns()
    
    sock.send(out_pkt)
    
#     times_1.append(end_1 - start_1)
#     times_2.append(end_2 - start_2)

In [ ]:
sock.close()

In [ ]:
print(np.average(times_1))
print(np.average(times_2))